In [1]:
import tensorflow as tf
from google.colab import drive
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Mount Google Drive
drive.mount('/content/drive')

# Clone the TensorFlow Models repository
!git clone https://github.com/tensorflow/models.git
!apt-get install protobuf-compiler
!cd models/research && protoc object_detection/protos/*.proto --python_out=.
!pip install .

# Import necessary libraries from Object Detection API
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# Load the model
PATH_TO_SAVED_MODEL = "PATH_TO_YOUR_SAVED_MODEL_DIR/saved_model"  # Update this path
model = tf.saved_model.load(PATH_TO_SAVED_MODEL)

# Load the label map
PATH_TO_LABELS = "PATH_TO_YOUR_LABEL_MAP.pbtxt"  # Update this path
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Function to load images from a folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

# Load your images
pothole_images = load_images_from_folder("/content/drive/MyDrive/Dataset/test/Pothole")
non_pothole_images = load_images_from_folder("/content/drive/MyDrive/Dataset/test/Plain")

# Run inference on a single image
def run_inference_for_single_image(image):
    image = np.asarray(image)
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis, ...]

    # Run inference
    output_dict = model(input_tensor)

    # All outputs are batches
    output_dict = {key: value.numpy() for key, value in output_dict.items()}
    output_dict['num_detections'] = int(output_dict['num_detections'][0])

    # Handle models with masks:
    output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.int64)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]

    return output_dict

# Visualize results
def show_inference(image):
    output_dict = run_inference_for_single_image(image)

    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8)

    return image

# Display results using Matplotlib
def display_results(images):
    for image in images:
        result_image = show_inference(image)
        result_image = cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB)  # Convert to RGB for displaying
        plt.figure(figsize=(12, 8))
        plt.imshow(result_image)
        plt.axis('off')  # Hide axes
        plt.show()  # Display image

# Display detection results for both pothole and non-pothole images
display_results(pothole_images)
display_results(non_pothole_images)


Mounted at /content/drive
Cloning into 'models'...
remote: Enumerating objects: 99697, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 99697 (delta 51), reused 37 (delta 37), pack-reused 99624 (from 3)
Receiving objects: 100% (99697/99697), 626.59 MiB | 28.37 MiB/s, done.
Resolving deltas: 100% (72561/72561), done.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
protobuf-compiler is already the newest version (3.12.4-1ubuntu7.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


ModuleNotFoundError: No module named 'object_detection'